# Pessimistic Neighbourhood Aggregation for States in Reinforcement Learning

Author: Maleakhi Agung Wijaya  
Supervisor: Marcus Hutter, Sultan Javed Majeed  
Date Created: 21/12/2017

## Mountain Car Environment

In [58]:
import math
class MountainCarEnvironment:
    '''Implementation of Sutton & Barto (1998) Mountain Car Problem environment.'''
    velocity_boundaries = (-0.07, 0.07)
    position_boundaries = (-1.2, 0.6)  
   
    # Constructor for MountainCarEnvironment
    # Input: agent for the MountainCarEnvironment
    # Output: MountainCarEnvironment object
    def __init__(self, car):
        self.car = car
        self.reset()
        
    # Compute next state (feature)
    # Output: (new velocity, new position)
    def nextState(self):
        # Get current state (velocity, position) and the action chosen by the agent
        velocity = self.car.state[0]
        position = self.car.state[1]
        action = self.car.doAction()
        
        # Calculate the new velocity and new position
        velocity += action * 0.001 + math.cos(3*position) * -0.0025
        position += + velocity
        
        new_state = (velocity, position)
        return(new_state)
    
    # Reset to the initial state    
    def reset(self):
        self.car.state[0] = 0.0
        self.car.state[1] = -0.5
        
    # Give reward for each of the chosen action, depending on what the next state that the agent end up in
    # Output: terminal state = 0, non-terminal state = -1
    def calculateReward(self):
        # Get current position of the agent
        position = self.car.state[1]
        
        # Determine the reward given
        if (position >= 0.6):
            return(0)
        else:
            return(-1)
        

## Car Agent

In [ ]:
class Agent:
    '''Implementation of agent (car) that will be used in the Mountain Car Environment'''
    initial_velocity = 0.0
    initial_position = -0.5
    
    # Constructor
    # Input: algorithm (class of algorithm that are used as the based method for our agent)
    def __init__(self, algorithm):
        self.state = [initial_velocity, initial_position]
        self.algorithm = algorithm
    
    # Allow car to choose action
    # Output: -1 (left), 0 (neutral), 1 (right)
    def doAction(self):
        # TODO: epsilon greedy method for choosing action. For now, just always choose right
        return(1)

## kNN-TD Algorithm

In [ ]:
import random
import math
class KNN:
    '''kNN TD algorithm approach for reinforcement learning.
       Theory based on Jose Antonio Martin H., Javier de Lope, and Dario Maravall (The kNN-TD Reinforcement Learning Algorithm)
       with some modification. '''
    
    # Constructor
    # Input: the number of k-neighbours, size of the stored Q value
    def __init__(self, k, size):
        self.k = k # how many neighbours
        self.q_storage = [] # storage of the previous q values
        
        # Initialise the storage with random point 
        for i in range(size):
            initial_value = 0
            initial_state = [random.uniform(-0.07, 0.07), random.uniform(-1.2, 0.6)]
            
            # Each data on the array will consist of state, action pair + value
            data = {"state": initial_state, "value": initial_value, "action": initial_action}
            self.q_storage.append(data)
    
    # Stardise feature vector given
    # Input: feature vector to be standardised
    # Output: standardised feature vector
    def standardiseState(self, state):
        standardised_state = []
        standardised_velocity = 2 * ((state[0]+0.07) / (0.07+0.07)) - 1
        standardised_position = 2 * ((state[1]+1.2) / (0.6+1.2)) - 1
        standardised_state.append(standardised_velocity)
        standardised_state.append(standardised_position)
        
        return(standardised_state)
    
    # Calculate Euclidean distance between 2 standardised feature vectors
    # Input: 2 feature vectors
    # Output: distance between them
    def calculateDistance(self, vector1, vector2):
        return(math.sqrt((vector1[0]-vector2[0])**2 + (vector1[1]-vector2[1])**2))
    
    # Apply the kNN algorithm for feature vector and store the data point on the neighbours array
    # Input: feature vector of current state
    def kNN_TD(self, state):
        # Calculate k neighbour for the current state and put the value into neighbours and calculate the weight for each
        temp = [] # array  consisting of tuple (distance, original index, weight)
        neighbours = []
        value_neighbours = [] # array that are used to compute approximation of q value
        
        # Get the standardised version of state
        standardised_state = stardiseState(state)
        
        # Loop through every element in the storage array
        for i in range(len(q_storage)):
            data = q_storage[i]
            vector_2 = data["state"]
            distance = calculateDistance(standardised_state, vector_2)
            index = i
            weight = 1 / (1 + distance**2)
            
            # Create the tuple and append that to temp
            temp.append(tuple((distance, index, weight)))
        
        # After we finish looping through all of the point and calculating the standardise distance,
        # Sort the tuple based on the distance and only take k of it and append that to the neighbours array
        sorted_temp = sorted(temp, key=lambda x: x[0])
        for i in range(self.k):
            neighbours.append(sorted_temp[i]) # now the neighbours array consist of k closest neighbour
            value_neighbours.append(q_storage[sorted_temp[i][1]]) # Get the real data using the original index that we have save
            
        # As we have the neighbours (containing weight) and also value_neighbour (containing values), we can compute an estimate
        # of t+1 state, action pair
            
            
    
    